In [21]:
import json
import os
from PIL import Image
import random
import shutil

In [22]:
json_path = "train_bbox_annotations.json"
input_img_folder = "train_bbox_images"
output_img_folder = "images_jpg"
label_output_folder = "labels"

In [23]:
# Создание выходных папок
os.makedirs(output_img_folder, exist_ok=True)
os.makedirs(label_output_folder, exist_ok=True)

# Загрузка аннотаций
with open(json_path, "r") as f:
    data = json.load(f)

# Классы
class_map = {"vacant_lot": 0}

for image_data in data["images"]:
    file_name = image_data["file_name"]  # .tif
    base_name = os.path.splitext(file_name)[0]

    # Пути
    tif_path = os.path.join(input_img_folder, file_name)
    output_tif_path = os.path.join(output_img_folder, file_name)
    label_path = os.path.join(label_output_folder, base_name + ".txt")

    # Копирование .tif без преобразования
    try:
        img = Image.open(tif_path)
        img.save(output_tif_path)  # сохраняем как есть (tif)
    except Exception as e:
        print(f"❌ Ошибка при сохранении {file_name}: {e}")
        continue

    # Размеры изображения
    img_width = image_data["width"]
    img_height = image_data["height"]
    annotations = image_data["annotations"]

    yolo_lines = []

    # Обработка аннотаций
    for ann in annotations:
        class_name = ann["class"]
        bbox = ann["bbox"]  # [x, y, width, height]
        x, y, w, h = bbox

        x_center = x + w / 2
        y_center = y + h / 2

        # Нормализация
        x_center /= img_width
        y_center /= img_height
        w /= img_width
        h /= img_height

        class_id = class_map.get(class_name, 0)
        yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

    # Сохранение YOLO-антотации
    with open(label_path, "w") as out_f:
        out_f.write("\n".join(yolo_lines))

print("✅ Готово: .tif изображения и аннотации сохранены без конвертации.")


✅ Готово: .tif изображения и аннотации сохранены без конвертации.


In [24]:
# Параметры
image_ext = ".tif"  # или .tif если не конвертировал
image_dir = "images_jpg"
label_dir = "labels"
output_dir = "dataset"
split_ratio = 0.965  # 80% train, 20% val

# Получаем все изображения и их базовые имена
images = [f for f in os.listdir(image_dir) if f.endswith(image_ext)]
random.shuffle(images)

split_index = int(len(images) * split_ratio)
train_images = images[:split_index]
val_images = images[split_index:]

# Вспомогательная функция копирования
def copy_files(image_list, split):
    os.makedirs(os.path.join(output_dir, "images", split), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels", split), exist_ok=True)

    for img_name in image_list:
        base = os.path.splitext(img_name)[0]
        label_name = base + ".txt"

        # Пути
        src_img = os.path.join(image_dir, img_name)
        src_lbl = os.path.join(label_dir, label_name)

        dst_img = os.path.join(output_dir, "images", split, img_name)
        dst_lbl = os.path.join(output_dir, "labels", split, label_name)

        # Копируем
        shutil.copyfile(src_img, dst_img)

        if os.path.exists(src_lbl):
            shutil.copyfile(src_lbl, dst_lbl)
        else:
            print(f"⚠️ Нет аннотации для: {img_name}")

# Копируем
copy_files(train_images, "train")
copy_files(val_images, "val")

print("✅ Датасет разделён на train и val")


✅ Датасет разделён на train и val


In [1]:
from ultralytics import YOLO
import yaml
import os
import matplotlib.pyplot as plt
import cv2

In [ ]:
with open('hyp.my_aug.yaml', 'r') as f:
    hyp_params = yaml.safe_load(f)

model = YOLO('yolo11s.pt')

model.train(
    data='/home/y.komarevtsev/sorevnovanie/data.yaml',
    imgsz=640,       # Уменьшение разрешения
    batch=16,
    epochs=300,      # Количество эпох (можно увеличить по мере необходимости)
    patience=50,
    freeze=10,  # <- это ключевая строка
    **hyp_params
)


New https://pypi.org/project/ultralytics/8.3.134 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.130 🚀 Python-3.10.16 torch-2.7.0+cu126 CUDA:0 (Tesla V100-SXM2-32GB, 32494MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=0.1, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.8, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/y.komarevtsev/sorevnovanie/data.yaml, degrees=5.0, deterministic=True, device=None, dfl=2.0, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.005, hsv_s=0.4, hsv_v=0.2, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train16, nbs=64, nms=Fal

train: Scanning /home/y.komarevtsev/sorevnovanie/dataset/labels/train.cache... 579 images, 0 backgrounds, 0 corrupt: 100%|██████████| 579/579 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1266.6±779.3 MB/s, size: 549.5 KB)


val: Scanning /home/y.komarevtsev/sorevnovanie/dataset/labels/val.cache... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<?, ?it/s]


Plotting labels to runs/detect/train16/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train16
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         21         99      0.364      0.414      0.347      0.228



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     39/300      2.64G    0.01821      1.908      1.869        107        640:  92%|█████████▏| 34/37 [00:02<00:00, 12.66it/s]

     39/300      2.64G    0.01827      1.935      1.872         23        640: 100%|██████████| 37/37 [00:02<00:00, 12.65it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 11.21it/s]

                   all         21         99      0.386      0.515      0.343      0.223


In [30]:
import os
import cv2
import matplotlib.pyplot as plt

# Параметры: путь к датасету
images_dir = '/home/y.komarevtsev/sorevnovanie/dataset/images/train'           # Папка с .jpg или .png
labels_dir = '/home/y.komarevtsev/sorevnovanie/dataset/labels/train'           # Папка с .txt файлами аннотаций
class_names = ['vacant_lot']  # Замените на реальные имена классов

# Функция отрисовки боксов
def draw_boxes(image_path, label_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w, _ = img.shape

    with open(label_path, 'r') as f:
        for line in f:
            cls, x, y, bw, bh = map(float, line.strip().split())
            # YOLO -> (x_center, y_center, width, height) в относительных координатах
            x1 = int((x - bw / 2) * w)
            y1 = int((y - bh / 2) * h)
            x2 = int((x + bw / 2) * w)
            y2 = int((y + bh / 2) * h)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(img, class_names[int(cls)], (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1)

    return img

# Визуализация нескольких изображений
def show_samples(n=5):
    images = sorted([f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))])
    for img_file in images[:n]:
        image_path = os.path.join(images_dir, img_file)
        label_path = os.path.join(labels_dir, os.path.splitext(img_file)[0] + '.txt')
        if not os.path.exists(label_path):
            continue
        img_with_boxes = draw_boxes(image_path, label_path)
        plt.figure(figsize=(8, 6))
        plt.imshow(img_with_boxes)
        plt.title(img_file)
        plt.axis('off')
        plt.show()

# 🔁 Показываем первые 5
show_samples(n=5)